In [9]:
create table PP_OAP_SING_ZHEWEI_T.wz_cou_rate_data1 as (
    sel 
    rcvr_id,
    pmt_start_date,
    sndr_country,
    count(*) as coun_num
    from PP_OAP_SING_ZHEWEI_T.wz_country_data1
    where trans_id > 0 
    group by 1,2,3
)with data primary index(rcvr_id);
------------------------------------------------------------------------------------------
call pp_monitor.drop_table('PP_OAP_SING_ZHEWEI_T.wz_cou_rate_temp');
create table PP_OAP_SING_ZHEWEI_T.wz_cou_rate_temp as (
    sel
    rcvr_id,
    pmt_start_date,
    sum(coun_num) as daily_tot_num
    from PP_OAP_SING_ZHEWEI_T.wz_cou_rate_data1
    group by 1, 2
)with data primary index(rcvr_id);
------------------------------------------------------------------------------------------
call pp_monitor.drop_table('PP_OAP_SING_ZHEWEI_T.wz_cou_rate_data2');
create table PP_OAP_SING_ZHEWEI_T.wz_cou_rate_data2 as (
    sel t1.rcvr_id,
    t1.pmt_start_date,
    t1.sndr_country,
    t1.coun_num * 1.0000000 / t2.daily_tot_num as coun_rate
    from PP_OAP_SING_ZHEWEI_T.wz_cou_rate_data1 t1
    left join PP_OAP_SING_ZHEWEI_T.wz_cou_rate_temp t2
    on t1.rcvr_id = t2.rcvr_id and t1.pmt_start_date = t2.pmt_start_date
)with data primary index(rcvr_id);

HTML(value='')

[]

In [13]:
--sel top 100 * from PP_OAP_SING_ZHEWEI_T.wz_cou_rate_data1 where rcvr_id = '1932592284603123791' and pmt_start_date = '2018-04-14'
create table PP_OAP_SING_ZHEWEI_T.wz_cou_rate_data3 as (
    sel 
    rcvr_id,
    pmt_start_date,
    sndr_country,
    count(*) as coun_num
    from PP_OAP_SING_ZHEWEI_T.wz_case2_march_1
    where trans_id > 0 
    group by 1,2,3
)with data primary index(rcvr_id);
------------------------------------------------------------------------------------------
call pp_monitor.drop_table('PP_OAP_SING_ZHEWEI_T.wz_cou_rate_temp');
create table PP_OAP_SING_ZHEWEI_T.wz_cou_rate_temp as (
    sel
    rcvr_id,
    pmt_start_date,
    sum(coun_num) as daily_tot_num
    from PP_OAP_SING_ZHEWEI_T.wz_cou_rate_data3
    group by 1, 2
)with data primary index(rcvr_id);
------------------------------------------------------------------------------------------
call pp_monitor.drop_table('PP_OAP_SING_ZHEWEI_T.wz_cou_rate_data2');
create table PP_OAP_SING_ZHEWEI_T.wz_cou_rate_data4 as (
    sel t1.rcvr_id,
    t1.pmt_start_date,
    t1.sndr_country,
    t1.coun_num * 1.0000000 / t2.daily_tot_num as coun_rate
    from PP_OAP_SING_ZHEWEI_T.wz_cou_rate_data3 t1
    left join PP_OAP_SING_ZHEWEI_T.wz_cou_rate_temp t2
    on t1.rcvr_id = t2.rcvr_id and t1.pmt_start_date = t2.pmt_start_date
)with data primary index(rcvr_id);

--call pp_monitor.drop_table('PP_OAP_SING_ZHEWEI_T.wz_case2_merged_table');
create table PP_OAP_SING_ZHEWEI_T.wz_cou_merged_data as
(SELECT * FROM PP_OAP_SING_ZHEWEI_T.wz_cou_rate_data2
UNION
SELECT * FROM PP_OAP_SING_ZHEWEI_T.wz_cou_rate_data4)
with data primary index(rcvr_id);

HTML(value='')

[]

In [9]:
create table PP_OAP_SING_ZHEWEI_T.wz_cou_rate_data5 as (
    sel 
    a.rcvr_id,
    a.pmt_start_date,
    coalesce(a.sndr_country, b.sndr_country) as full_coun,
    --a.sndr_country, --as his_coun,
    a.coun_rate,
    b.sndr_country as his_country,
    --b.coun_rate as his_rate
    --b.pmt_start_date as his_date
    avg(b.coun_rate) as his_rate
    from PP_OAP_SING_ZHEWEI_T.wz_cou_merged_data a
    full outer join PP_OAP_SING_ZHEWEI_T.wz_cou_merged_data b
    on a.rcvr_id = b.rcvr_id 
    where a.pmt_start_date between '2018-04-01' and '2018-04-03'
and b.pmt_start_date between a.pmt_start_date-30 and a.pmt_start_date-1
group by 1,2,3, 4, 5
)with data primary index(rcvr_id, pmt_start_date);

HTML(value='')

DatabaseError: (3156, '[HY000] [Teradata][ODBC Teradata Driver][Teradata Database] Request aborted by TDWM. Exception criteria exceeded:   AMP I/O Skew. ')

DatabaseError: (3156, '[HY000] [Teradata][ODBC Teradata Driver][Teradata Database] Request aborted by TDWM. Exception criteria exceeded:   AMP I/O Skew. ')

In [12]:
--call pp_monitor.drop_table('PP_OAP_SING_ZHEWEI_T.wz_cou_merged_filtered_data');
create table PP_OAP_SING_ZHEWEI_T.wz_cou_merged_filtered_data AS(
select t1.rcvr_id,
    t1.pmt_start_date,
    t1.sndr_country,
    t1.coun_rate
    from PP_OAP_SING_ZHEWEI_T.wz_cou_merged_data t1
    left join PP_OAP_SING_ZHEWEI_T.template t2
    on t1.rcvr_id = t2.rcvr_id
    left join PP_OAP_SING_ZHEWEI_T.template1 t3
    on t1.rcvr_id = t3.rcvr_id
    where t3.rcvr_id is Null and 
        t2.rcvr_id is Null
)with data primary index(rcvr_id);

HTML(value='')

rcvr_id,pmt_start_date,full_coun,coun_rate,his_country,his_rate
1361466912107315613,2018-04-01,AU,1.0000000,SG,0.1145299
1361466912107315613,2018-04-01,AU,1.0000000,AU,0.9885470
1361466912107315613,2018-04-02,AU,1.0000000,AU,0.9885470
1361466912107315613,2018-04-02,AU,1.0000000,SG,0.1145299
1361466912107315613,2018-04-03,BR,0.1250000,AU,0.9885470
1361466912107315613,2018-04-03,BR,0.1250000,SG,0.1145299
1361466912107315613,2018-04-03,AU,0.8750000,AU,0.9885470
1361466912107315613,2018-04-03,AU,0.8750000,SG,0.1145299
1361466912107315613,2018-04-04,AU,1.0000000,BR,0.1250000
1361466912107315613,2018-04-04,AU,1.0000000,SG,0.1145299


In [16]:
call pp_monitor.drop_table('PP_OAP_SING_ZHEWEI_T.wz_LM_data');
create table PP_OAP_SING_ZHEWEI_T.wz_LM_data AS(
select t1.rcvr_id,
    t1.pmt_start_date,
    t1.sndr_country,
    t1.coun_rate
    from PP_OAP_SING_ZHEWEI_T.wz_cou_merged_filtered_data t1
    right join PP_OAP_SING_ZHEWEI_T.wz_LM t2
    on t1.rcvr_id = t2.rcvr_id
)with data primary index(rcvr_id);

HTML(value='')

HTML(value='')

TabView(children=(VerticalBox(children=(QGridCustomWidget(grid_options={'highlightSelectedCell': True, 'filter…